## P300 Speller Letter Classification Training ver1: SWLDA
- P300 Speller BCI (Brain-Computer Interface) 시스템에서 SWLDA(Stepwise Linear Discriminant Analysis) 기반으로 학습 및 테스트를 수행하여, EEG(뇌파) 데이터로부터 스펠링하려는 문자를 인식하는 과정정
- 목적: EEG 기반 P300 Speller 데이터에서 문자를 분류하는 모델 학습 및 평가
    - BCI2000 데이터셋을 기반으로 55명의 피험자에 대해
    - EEG 신호로부터 문자 인식 정확도를 훈련 및 테스트하여
    - 반복 횟수(repetition)에 따른 정확도 향상 여부를 분석하고
    - 전체 평균 및 피험자별 정확도를 시각화하고 저장하는 데 목적이 있다.
- 데이터: BCI2000 (32채널 Biosemi2, 총 55명 피험자)


In [ ]:
clear; clc; # 작업 환경 초기화 (변수 및 콘솔 클리어)

In [ ]:
# 사용할 EEG 채널 인덱스 설정
ch = 1:32;

# 철자 매트릭스 정의 (6x6 자판 구성)
spellermatrix = ...
    ['A', 'B', 'C', 'D', 'E', 'F', ...
    'G', 'H', 'I', 'J', 'K', 'L', ...
    'M', 'N', 'O', 'P', 'Q', 'R', ...
    'S', 'T', 'U', 'V', 'W', 'X', ...
    'Y', 'Z', '1', '2', '3', '4', ...
    '5', '6', '7', '8', '9', '_'];

% 실험 설정 정보
Configs_P3speller = struct('seq_code', 1:12, 'full_repeat', 15, ...
    'spellermatrix', spellermatrix);

% EEG 전처리 파라미터 정의
Params_P3speller = struct('freq', [0.5 10], 'frame', [0 600], ...
    'baseline', [-200 0], 'select_ch', 1:32);

1. 학습 단계
    - 각 피험자별로 학습 데이터를 불러오고,
    - SWLDA 모델을 학습시켜 피험자의 EEG 신호로부터 문자를 분류할 수 있도록 한다.
2. 테스트 단계
    - 테스트 세트를 사용하여 모델 성능을 평가하고,
    - 반복 횟수에 따라 문자 인식 정확도가 어떻게 달라지는지를 측정한다.
3. 시각화 및 저장
    - 반복 횟수별, 피험자별 정확도를 박스 차트 및 막대그래프로 시각화하고,
    - 결과를 .mat 파일로 저장한다.

In [ ]:
% 55명 피험자에 대해 반복 수행
for nsb=1:55
    fname = sprintf('./data/s%02d.mat', nsb);  % 파일 경로 생성
    EEG = load(fname);  % EEG 데이터 로드
    eeg_train = EEG.train;  % 학습용 데이터 분리

    % ------------------------ 학습 데이터 처리 ------------------------ %
    feat_train = [];  % 특징 벡터 저장용 변수
    label_train = [];  % 정답 레이블 저장용 변수
    for nRun = 1:length(eeg_train)
        cur_eeg = eeg_train{nRun};
        data = cur_eeg.data(ch,:);  % 원하는 채널만 추출
        cur_eeg.data = data;  % 데이터 갱신

        % 전처리 및 특징 추출
        [feature_set, label_set] = preproc_P3speller(cur_eeg, Params_P3speller, 'labeled');
        feat_train = cat(1, feat_train, feature_set);
        label_train = cat(1, label_train, label_set);
    end

    % ---------------------- SWLDA 모델 학습 ------------------------ %
    rd = shuffle([feat_train label_train]);  % 데이터를 섞음
    feat_train = rd(:, 1:end-1);  % 특징 벡터
    label_train = rd(:, end);  % 정답 레이블
    mdl_SWLDA = trainSWLDA(feat_train, label_train, 0.08, 60);  % SWLDA 학습

    predictor = predictSWLDA(mdl_SWLDA, feat_train, 'label');  % 예측 수행
    acc_binary_epoch = sum(predictor==label_train) / length(label_train);  % 정확도 계산
    fprintf('P300 speller train epoch classification accuracy is %.2f%%\n', 100*acc_binary_epoch);

    % ------------------------ 테스트 데이터 처리 ---------------------- %
    eeg_test = EEG.test;
    for nRun = 1:length(eeg_test)
        cur_eeg = eeg_test{nRun};
        word_len = cur_eeg.nbTrials / (length(Configs_P3speller.seq_code)*Configs_P3speller.full_repeat);

        data = cur_eeg.data(ch,:);  % 분석 대상 채널 추출
        cur_eeg.data = data;
        [feat_test, label_train] = preproc_P3speller(cur_eeg, Params_P3speller, 'labeled');

        rd = shuffle([feat_test label_train]);
        feat_test = rd(:, 1:end-1);
        label_train = rd(:, end);

        predictor = predictSWLDA(mdl_SWLDA, feat_test, 'label');
        acc_binary_epoch = sum(predictor==label_train) / length(label_train);  % (출력 안함)

        % ---- repetition 기반 문자 인식 정확도 계산 ---- %
        markers_seq = cur_eeg.markers_seq;
        letter_idx = find(ismember(cur_eeg.markers_seq, Configs_P3speller.seq_code));
        feat_unknown = preproc_P3speller(cur_eeg, Params_P3speller, 'unknown');

        % 문자인식 수행 및 결과 저장
        ans_letters = detect_letter_P3speller(mdl_SWLDA, word_len, feat_unknown, ...
            cur_eeg.text_to_spell, letter_idx, markers_seq, Configs_P3speller);

        acc_on_repetition(nsb, nRun, :) = ans_letters.acc_on_repetition;
        correct_on_repetition(nsb, nRun, :) = ans_letters.correct_on_repetition;

        fprintf('User answer: %s (%d/%d), accuracy: %.2f\n', ...
            ans_letters.text_result, correct_on_repetition(nsb, nRun, 15), word_len, acc_on_repetition(nsb, nRun, 15));
    end

    fprintf('\nOverall: %d/%d, accuracy: %.2f\n\n', sum(correct_on_repetition(nsb, :, Configs_P3speller.full_repeat), 2), ...
        word_len*nRun, sum(correct_on_repetition(nsb, :, Configs_P3speller.full_repeat), 2)/(word_len*nRun));
end

In [ ]:
% 결과 저장
save('Total_performance.mat', 'acc_on_repetition', 'correct_on_repetition');

%% 결과 분석 및 시각화

load('Total_performance.mat');  % 저장된 결과 불러오기

% correct_on_repetition의 사이즈 확인 (55명 x 횟수 x 반복)
disp(size(correct_on_repetition));

total_corrected_repeat = squeeze(sum(correct_on_repetition, 2));  % 각 repetition마다 총 맞춘 개수

% 반복별 정확도 계산 (총 28문자)
total_acc_repeat = total_corrected_repeat / 28;

avgd_acc = mean(total_acc_repeat, 1);  % 반복 횟수별 평균 정확도

% Boxplot 시각화 (repetition 수에 따른 정확도 분포)
figure, 
h=boxchart(total_acc_repeat);
pbaspect([2 1 1]);
set(gca, 'fontsize', 13);
xlabel('The nubmer of repetitions');
ylabel('Letter detection accuracy');
hold on;
box on;
plot(avgd_acc, '-o');  % 평균값 선 그래프

% 막대 그래프 (피험자별 최종 반복 정확도)
sky_boxchart = [0 0.4470 0.7410];
figure,
bar(total_acc_repeat(:, end), ...
    'FaceColor', sky_boxchart, 'FaceAlpha', 0.2, ...
    'EdgeColor', sky_boxchart, 'Linewidth', 1.2); 
set(gca, 'fontsize', 13);
xlabel('Subjects');
ylabel('Offline letter detection accuracy');
ylim([0.4 1]);
xlim([0 56]);
pbaspect([2 1 1]);
